<a href="#1-Definition" style="margin-left: 0px;">1 Definition</a>   
<a href="#2-Error" style="margin-left: 0px;">2 Error</a>  
<a href="#3-Stopping-Criteria-in-Decision-Trees" style="margin-left: 0px;">3 Stopping Criteria in Decision Trees</a>  
<a href="#4-Implementation" style="margin-left: 0px;">4 Implementation</a>  

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

### 1 Definition

Дерево решений - метод машинного обучения, где target может быть вещественное число или категория. Структура дерева представляет собой «листья» и «ветки». На "ветках" указывается набор признаков, на "листьях" указываются показатели выборки.

В решающих деревьях мы используем функцию CART в качестве функции потерь.

### 2 Error  

**CART Classification and Regression Tree**

Loss Function (функция потерь), Error function - в математике это есть такая конкретная функция    
$J(k,t_{k}) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}$ -> **min!!!**

#### 1) Classification task (Задача классификации):  
 
A) Gini coefficient (нечистоты Джини - Gini impurity):  
$G_{i} = 1 - \sum\limits_{i=1}^{n}P_{i,k}^{2}$, where $P_{i,k}$ - is the proportion of class k in node i.  


Пример: 


|        | 1   | 0   | all |
|--------|-----|-----|-----|
| left   | 136 | 80  | 216 |
| right  | 205 | 469 | 675 |

In [2]:
left_gini = 1 - (136/216)**2 - ((216-136)/216)**2

print("left_gini:", left_gini)

right_gini = 1 - (206/675)**2 - ((675-206)/675)**2

print("right_gini: ", right_gini)

CART = (216/(216+675))*left_gini + (675/(216+675))*right_gini

print("CART: ",CART)

left_gini: 0.46639231824417005
right_gini:  0.42409437585733895
CART:  0.43434842249657074


B) Entropy:  
$H_{i} = -\sum\limits_{i=1}^{n}P_{i,k}log(P_{i,k})$  
If $H_{i}$ = 0 => Good split  

Пример: 


|        | 1   | 0   | all |
|--------|-----|-----|-----|
| left   | 136 | 80  | 216 |
| right  | 205 | 469 | 675 |

In [3]:
p1_left = 136 / 216
p0_left = 80 / 216
left_entropy = -p1_left * np.log2(p1_left) - p0_left * np.log2(p0_left)
print("left_entropy:", left_entropy)

p1_right = 205 / 675
p0_right = 469 / 675
right_entropy = -p1_right * np.log2(p1_right) - p0_right * np.log2(p0_right)
print("right_entropy:", right_entropy)

total = 216 + 675
CART_entropy = (216 / total) * left_entropy + (675 / total) * right_entropy
print("CART_entropy:", CART_entropy)

left_entropy: 0.9509560484549725
right_entropy: 0.8871326506637249
CART_entropy: 0.9026049895222091


#### 2) Regression analysis (Регрессионный анализ):   

$MSE_{node} = \frac{1}{m_{node}}\sum\limits_{i∈node}(\hat{y}_{node}-y^{(i)})^{2}$     
$\hat{y}_{node} = \frac{1}{m_{node}}\sum\limits_{i∈node}y^{(i)}$   

Пример:  

|        | Count | Mean |
|--------|-------|------|
| Left   | 216   | 5.10 |
| Right  | 675   | 6.28 |
| Total  | 891   | —    |


In [4]:
n_left = 216
n_right = 675
n_total = n_left + n_right

var_left = 0.04
var_right = 0.09

cart_mse = (n_left / n_total) * var_left + (n_right / n_total) * var_right
print("CART (MSE):", cart_mse)

CART (MSE): 0.07787878787878788


### 3 Stopping Criteria in Decision Trees
Критерии останова в дереве решений

1) Максимальная глубина  
2) Минимальное количество объектов в листе
3) Если ни один признак не может разделить данные лучше (например, все объекты в узле одинаковые или разбиение даёт такое же Джинни) 
4) Слишком маленькие подгруппы после разбиения (min_samples_split)

### 4 Implementation

#### Classification

| №  | Название колонки | Тип данных       | Описание |
|----|------------------|------------------|----------|
| 1  | `PassengerId`    | `int`            | Уникальный ID пассажира |
| 2  | `Survived`       | `int` (0/1)      | Выжил (1) или нет (0) — **таргет для предсказания** |
| 3  | `Pclass`         | `int` (1-3)      | Класс билета:<br>• 1 = 1-й класс<br>• 2 = 2-й класс<br>• 3 = 3-й класс |
| 4  | `Name`           | `string`         | Имя пассажира (включая титул, например, "Mr.", "Miss") |
| 5  | `Sex`            | `string`         | Пол (`"male"` или `"female"`) |
| 6  | `Age`            | `float`          | Возраст (есть пропуски — `NaN`) |
| 7  | `SibSp`          | `int`            | Количество siblings (братья/сёстры) + spouses (супруги) на борту |
| 8  | `Parch`          | `int`            | Количество parents (родители) + children (дети) на борту |
| 9  | `Ticket`         | `string`         | Номер билета (может содержать буквы и цифры) |
| 10 | `Fare`           | `float`          | Стоимость билета |
| 11 | `Cabin`          | `string`         | Номер каюты (много пропусков — `NaN`) |
| 12 | `Embarked`       | `string`         | Порт посадки:<br>• `C` = Cherbourg<br>• `Q` = Queenstown<br>• `S` = Southampton |

In [5]:
data = pd.read_csv(r"../00 Data/titanic.csv")

print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Замена пропусков в Age медианным значением
data['Age'] = data['Age'].fillna(data['Age'].median())

# Замена пропусков в Embarked модой
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# Семейный размер
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

# Одиночка (1, если FamilySize = 1)
data['IsAlone'] = (data['FamilySize'] == 1).astype(int)

# Encoder
encoder = TargetEncoder(cols=['Sex', 'Embarked', 'FamilySize'])

data[['Sex', 'Embarked', 'FamilySize']] = encoder.fit_transform(
    data[['Sex', 'Embarked', 'FamilySize']], 
    data['Survived']  # Целевая переменная
)

In [7]:
# StandardScaler для всех признаков
features = data.columns.drop('Survived')
data[features] = StandardScaler().fit_transform(data[features])

In [8]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,0,0.827377,-0.737695,-0.565736,0.432793,-0.473674,-0.502445,-0.539973,1.290322,-1.231645
1,1,-1.566107,1.355574,0.663861,0.432793,-0.473674,0.786845,2.044556,1.290322,-1.231645
2,1,0.827377,1.355574,-0.258337,-0.474545,-0.473674,-0.488854,-0.539973,-0.687981,0.811922
3,1,-1.566107,1.355574,0.433312,0.432793,-0.473674,0.420730,-0.539973,1.290322,-1.231645
4,0,0.827377,-0.737695,0.433312,-0.474545,-0.473674,-0.486337,-0.539973,-0.687981,0.811922


**1 build_tree**

1) Берет X, y    
2) Создаёт Node со своим value и class_counts = {0: 10, 1: 20}    
3) Находит лучшее разбиение (колонка, порог)    
4) Делит на 2 подвыборки X_left, X_right       
5) и далее рекурсивно идет в левую ветку и там проделывает тоже самое и в правую ветку и проделывает тоже самое (создаёт Node и находит лучшее разбиение)  
6) Останавливается когда:  
6.1) Если достигнута максимальная глубина дерева (depth >= max_depth)   
6.2) Если в узле осталось слишком мало объектов (len(y) < min_samples)   
6.3) Остаётся 1 класс
6.4) Доходит до конца

**2 search_best_threshold_for_one_column**

Берет сгруппированную таблицу и находит 2 параметра:  
    1) best_gini_for_one_column = Минимальный Джинни  
    2) best_threshold_for_devide_for_one_column = Лучший порог  

### Пример расчёта CART для одной переменной (Pclass)

In [9]:
column = 1 # Pclass

print("Уникальные значения Pclass: ",set(data.iloc[:, 1]))
train_small = pd.concat([data.iloc[:, 1], data["Survived"]], axis=1)

Уникальные значения Pclass:  {0.8273772438659699, -0.3693648410115938, -1.5661069258891576}


In [10]:
column = list(train_small.columns)[0]
train_small = train_small.groupby([column]).agg(['sum', 'count'])
train_small.columns = train_small.columns.droplevel(0)  # Убираем верхний уровень индекса

train_small = train_small.reset_index()
train_small = train_small[[column, 'sum', 'count']]
all_target = sum(train_small["sum"])
all_count = sum(train_small["count"])

train_small['left_target'] = train_small["sum"].cumsum()
train_small['left_count'] = train_small["count"].cumsum()
train_small['right_target'] = all_target - train_small['left_target']
train_small['right_count'] = all_count - train_small['left_count']
train_small['left_share'] = train_small['left_target'] / train_small['left_count']
train_small['right_share'] = train_small['right_target'] / train_small['right_count']

# left_gini = 1 - (p_0_left**2 + p_1_left**2) = 1 - (p_0_left**2 + (1-p_0_left)**2)

train_small['left_gini'] = 1 - (train_small["left_share"] ** 2 + (1 - train_small["left_share"]) ** 2) 
train_small['right_gini'] = 1 - (train_small["right_share"] ** 2 + (1 - train_small["right_share"]) ** 2)
train_small['CART'] = (train_small['left_gini'] * train_small['left_count']) / all_count + (train_small['right_gini'] * train_small['right_count']) / all_count

### Пример расчёта CART для одного разбиения

left_target - количество объектов в левом листе и target ==1  
left_count - количество объектов в левом листе    
left_target - количество объектов в право листе и target ==1  
right_count - количество объектов в правом листе  

In [11]:
train_small

,Pclass,sum,count,left_target,left_count,right_target,right_count,left_share,right_share,left_gini,right_gini,CART
0,-1.566107,136,216,136,216,206,675,0.629630,0.305185,0.466392,0.424094,0.434348
1,-0.369365,87,184,223,400,119,491,0.557500,0.242363,0.493387,0.367246,0.423875
2,0.827377,119,491,342,891,0,0,0.383838,NaN,0.473013,NaN,NaN


In [12]:
left_gini = 1 - (136/216)**2 - ((216-136)/216)**2

print("left_gini:", left_gini)

right_gini = 1 - (206/675)**2 - ((675-206)/675)**2

print("right_gini: ", right_gini)

CART = (216/(216+675))*left_gini + (675/(216+675))*right_gini

print("CART: ",CART)

left_gini: 0.46639231824417005
right_gini:  0.42409437585733895
CART:  0.43434842249657074


**3 search_best_threshold**

Находит:  
    1) column_for_devide - колонку лучшего разбиения  
    2) threshold_for_devide - порог для разбиения  

**4 predict_proba**  
1) Для каждого  объекта X(идем по строчке)    
2) Идёт  рекусривно по дереву (смотрим на колонку и порог в зависимости от этого и проваливаемся в левое или правое поддерево)  
3) Когда долшли что и право и левое поддерево это уже пусто, то добавляем value(вероятность класса)

**5 predict**  

Просто прохдим по X и применяем predict_proba и если >=0.5, то класс "1", иначе "0"

In [13]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None, class_counts=None):
        self.feature = feature # название колонки
        self.threshold = threshold # пороговое значение для разбиения
        self.left = left
        self.right = right
        self.value = value # среднее значение целевой переменной y в этом листе (y = [0, 1, 1, 0, 1], то value = 3/5 = 0.6)
        self.class_counts = class_counts  # Распределение классов для листа, например class_counts = {0: 10, 1: 20}
        self.gini = self.calculate_gini() if class_counts else None  # Вычисляем Gini, если есть class_counts
        
    def calculate_gini(self):
        if not self.class_counts:
            return None  # Если class_counts нет, возвращаем None

        total = sum(self.class_counts.values())  # Общее число элементов
        if total == 0:
            return 0  # Если узел пустой, Gini = 0

        # Вычисляем доли классов (0:30,1:10 -> proportions = [0.75,0.25])
        proportions = [count / total for count in self.class_counts.values()]

        # Индекс Gini: 1 - сумма(p^2) для всех классов
        # 1 - (0.75² + 0.25²) = 1 - (0.5625 + 0.0625) = 1 - 0.625 = 0.375
        gini = 1 - sum(p**2 for p in proportions)
        return gini
        
class My_Decision_Tree_Classifier:

    def __init__(self, max_depth, min_samples_split=0.000001, min_samples=10):
        self.max_depth = max_depth # максимальная глубина
        self.min_samples_split = min_samples_split # минимальное количество объектов в листе относительно всей выборки
        self.min_samples = min_samples # минимальное количество объектов в листе
        self.root = None

    def search_best_threshold_for_one_column(self, train_small):
        if len(set(train_small.iloc[:, 0])) == 1:
            return [10000, 10000]

        best_threshold_for_devide_for_one_column = 10000 # инициализируем лучшее разбиение для признака
        best_gini_for_one_column = 10000 # инициализируем лучший gini для признака

        column = list(train_small.columns)[0]
        train_small = train_small.groupby([column]).agg(['sum', 'count'])
        train_small.columns = train_small.columns.droplevel(0)  # Убираем верхний уровень индекса

        train_small = train_small.reset_index()
        train_small = train_small[[column, 'sum', 'count']]
        all_target = sum(train_small["sum"])
        all_count = sum(train_small["count"])

        train_small['left_target'] = train_small["sum"].cumsum()
        train_small['left_count'] = train_small["count"].cumsum()
        train_small['right_target'] = all_target - train_small['left_target']
        train_small['right_count'] = all_count - train_small['left_count']
        train_small['left_share'] = train_small['left_target'] / train_small['left_count']
        train_small['right_share'] = train_small['right_target'] / train_small['right_count']
        
        # left_gini = 1 - (p_0_left**2 + p_1_left**2) = 1 - (p_0_left**2 + (1-p_0_left)**2)
        
        train_small['left_gini'] = 1 - (train_small["left_share"] ** 2 + (1 - train_small["left_share"]) ** 2) 
        train_small['right_gini'] = 1 - (train_small["right_share"] ** 2 + (1 - train_small["right_share"]) ** 2)
        train_small['CART'] = (train_small['left_gini'] * train_small['left_count']) / all_count + (train_small['right_gini'] * train_small['right_count']) / all_count

        train_small.loc[train_small.index[-1], "CART"] = 10000

        train_small = train_small[(train_small["left_count"] >= all_count * self.min_samples_split) & (train_small["right_count"] >= all_count * self.min_samples_split)].reset_index(drop=True)
        train_small = train_small[(train_small["left_count"] >= self.min_samples) & (train_small["right_count"] >= self.min_samples)].reset_index(drop=True)

        if train_small.shape[0] > 0: # Проверяет, остались ли допустимые разбиения после фильтрации (train_small.shape[0] > 0
            index_of_min = train_small['CART'].idxmin() # Находит разбиение с минимальным значением CART

            best_gini_for_one_column = train_small["CART"].iloc[index_of_min]
            best_threshold_for_devide_for_one_column = train_small[column].iloc[index_of_min]

        return [best_threshold_for_devide_for_one_column, best_gini_for_one_column]

    def search_best_threshold(self, X, y):
        column_for_devide = None
        threshold_for_devide = None
        best_gini = 10000
        columns = list(X.columns)

        for column in range(X.shape[1]):
            train_small = pd.concat([X.iloc[:, column], y], axis=1)

            best_threshold_for_devide_for_one_column, best_gini_for_one_column = self.search_best_threshold_for_one_column(train_small)
            if best_gini_for_one_column == 10000:
                continue

            left_mask = X[columns[column]] <= best_threshold_for_devide_for_one_column
            right_mask = X[columns[column]] > best_threshold_for_devide_for_one_column

            min_samples = int(self.min_samples_split * len(y))
            left_size = len(y[left_mask])
            right_size = len(y[right_mask])

            if left_size >= min_samples and right_size >= min_samples:
                if best_gini > best_gini_for_one_column:
                    best_gini = best_gini_for_one_column
                    column_for_devide = columns[column]
                    threshold_for_devide = best_threshold_for_devide_for_one_column

        return [column_for_devide, threshold_for_devide]

    def build_tree(self, X, y, depth):
        # Если достигнута максимальная глубина дерева (depth >= max_depth).
        # Если в узле осталось слишком мало объектов (len(y) < min_samples
        if depth >= self.max_depth or len(y) < self.min_samples:
            value = y.mean() if y.nunique() > 1 else y.iloc[0] # считает  value как среднее, если уникальных значений 1, то берем это значение
            # Создаём узел с таким value и class_counts
            return Node(value=value, class_counts=y.value_counts().to_dict())

        # иначе находим лучшее разбиение
        column, threshold = self.search_best_threshold(X, y)
        
        # если колонка пустая
        if column is None:
            value = y.mean() if y.nunique() > 1 else y.iloc[0]
            # Создаём узел с таким value и class_counts
            return Node(value=value, class_counts=y.value_counts().to_dict())

        # формируем подвыбрки по лучшему разбиению
        left_mask = X[column] <= threshold
        right_mask = X[column] > threshold

        # строим левый и правые узлы
        left_child = self.build_tree(X[left_mask], y[left_mask], depth + 1)
        right_child = self.build_tree(X[right_mask], y[right_mask], depth + 1)

        # возвращаем узел
        return Node(feature=column, threshold=threshold, left=left_child, right=right_child)

    def fit(self, X, y):
        self.root = self.build_tree(X, y, depth=0)

    def predict_proba(self, X):
        # Создаем копию X
        X = X.copy()

        # Применяем дерево для каждого примера
        predictions = []
        for _, row in X.iterrows():
            node = self.root
            # Идем по дереву
            while node.left is not None and node.right is not None:
                if row[node.feature] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            # Находим вероятность для листа
            predictions.append(node.value)  # Среднее значение в этом листе

        return np.array(predictions)
    
    def predict(self, X):
        # Получаем вероятности классов с помощью predict_proba
        probabilities = self.predict_proba(X)
        # Преобразуем вероятности в классы (0 или 1)
        return (probabilities >= 0.5).astype(int)

In [14]:
%%time
my_clf = My_Decision_Tree_Classifier(max_depth=5)
my_clf.fit(data.iloc[:, 1:], data['Survived'])

CPU times: total: 1.38 s
Wall time: 1.38 s


In [15]:
%%time
my_y_proba = my_clf.predict_proba(data.iloc[:, 1:])

CPU times: total: 31.2 ms
Wall time: 36.2 ms


In [16]:
%%time
clf = DecisionTreeClassifier(max_depth=5,random_state=42)
clf.fit(data.iloc[:, 1:], data['Survived'])

CPU times: total: 0 ns
Wall time: 6.97 ms


DecisionTreeClassifier(max_depth=5, random_state=42)

In [17]:
%%time
y_proba = clf.predict_proba(data.iloc[:, 1:])[:,1]

CPU times: total: 0 ns
Wall time: 3.05 ms


In [18]:
# Вычисление AUC-ROC
AUC_my = roc_auc_score(data['Survived'].values, my_y_proba)
AUC_sclearn = roc_auc_score(data['Survived'].values, y_proba)

print(f"AUC-ROC (Custom Model): {AUC_my:.4f}")
print(f"AUC-ROC (Custom Model): {AUC_sclearn:.4f}")

AUC-ROC (Custom Model): 0.8874
AUC-ROC (Custom Model): 0.8987


#### Результат схожий

#### Регрессия

Здесь я просто беру "ребёнка" считаю его стандартное отклонение, то есть среднее значение это и есть его предсказание, далее считаю стандартное отелонение как коэффициент ошибки и подставляю его в формулу CART
$
\sigma =  \sqrt{\frac{1}{N} \sum_{i=1}^{N} (x_i - \mu)^2}
$

In [19]:
data = pd.read_csv(r"../00 Data/real estate.csv")

In [20]:
print(data.shape)
data.head()

(49352, 24)


,bathrooms,bedrooms,interest_level,price,Elevator,CatsAllowed,HardwoodFloors,DogsAllowed,Doorman,Dishwasher,...,LaundryinUnit,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
0,1.0,1,1,2400,0,1,1,1,0,1,...,0,0,0,1,0,0,0,0,0,0
1,1.0,2,0,3800,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1.0,2,1,3495,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
3,1.5,3,1,3000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,2795,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X = data.drop(['price'],axis=1)
y = data['price']

In [22]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None, class_counts=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.class_counts = class_counts  # Распределение классов для листа

class My_Decision_Tree_Regressor:

    def __init__(self, max_depth, min_samples_split=0.000001, min_samples=10):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples = min_samples
        self.root = None

    # Новый метод для вычисления стандартной ошибки
    def calculate_standard_error(self, y):
        return np.std(y) / np.sqrt(len(y))
    
    def search_best_threshold_for_one_column(self,train_small):
        if len(set(train_small.iloc[:, 0])) == 1:
            return [10000, 10000]

        best_threshold_for_devide_for_one_column = 10000
        best_diff_st_er_for_one_column = 10000

        column = list(train_small.columns)[0]
        column_y = list(train_small.columns)[1]

        # Группируем по столбцу и считаем суммарные и количественные показатели
        train_small_group = train_small.groupby([column]).agg(['count'])
        train_small_group.columns = train_small_group.columns.droplevel(0)  # Убираем верхний уровень индекса
        train_small_group = train_small_group.reset_index()
        train_small_group = train_small_group[[column, 'count']]
        all_count = sum(train_small_group["count"])

        # Считаем кумулятивные суммы для левой и правой части
        train_small_group['left_count'] = train_small_group["count"].cumsum()
        train_small_group['right_count'] = all_count - train_small_group['left_count']

        # Вычисляем стандартные ошибки для левой и правой части
        train_small_group['standart_error_left'] = train_small_group[column].apply(
        lambda threshold: self.calculate_standard_error(train_small[train_small[column] <= threshold].iloc[:, -1])
        )
        train_small_group['standart_error_right'] = train_small_group[column].apply(
            lambda threshold: self.calculate_standard_error(train_small[train_small[column] > threshold].iloc[:, -1])
        )

        # Добавляем столбец CART для разницы стандартных ошибок
        train_small_group['CART'] = (
            (train_small_group['standart_error_left'] * train_small_group['left_count']) / all_count +
            (train_small_group['standart_error_right'] * train_small_group['right_count']) / all_count
        )


        # Применяем фильтры по минимальному количеству элементов в группах
        train_small_group = train_small_group[
            (train_small_group['left_count'] >= all_count * self.min_samples_split) & 
            (train_small_group['right_count'] >= all_count * self.min_samples_split)
        ].reset_index(drop=True)

        train_small_group = train_small_group[
            (train_small_group['left_count'] >= self.min_samples) & 
            (train_small_group['right_count'] >= self.min_samples)
        ].reset_index(drop=True)

        # Находим минимальное значение разницы CART и порог
        if train_small_group.shape[0] > 0:
            index_of_min = train_small_group['CART'].idxmin()
            best_diff_st_er_for_one_column = train_small_group["CART"].iloc[index_of_min]
            best_threshold_for_devide_for_one_column = train_small_group[column].iloc[index_of_min]

        return [best_threshold_for_devide_for_one_column, best_diff_st_er_for_one_column]

    def search_best_threshold(self, X, y):
        column_for_devide = None
        threshold_for_devide = None
        best_gini = 10000
        columns = list(X.columns)

        for column in range(X.shape[1]):
            train_small = pd.concat([X.iloc[:, column], y], axis=1)

            best_threshold_for_devide_for_one_column, best_gini_for_one_column = self.search_best_threshold_for_one_column(train_small)
            if best_gini_for_one_column == 10000:
                continue

            left_mask = X[columns[column]] <= best_threshold_for_devide_for_one_column
            right_mask = X[columns[column]] > best_threshold_for_devide_for_one_column

            min_samples = int(self.min_samples_split * len(y))
            left_size = len(y[left_mask])
            right_size = len(y[right_mask])

            if left_size >= min_samples and right_size >= min_samples:
                if best_gini > best_gini_for_one_column:
                    best_gini = best_gini_for_one_column
                    column_for_devide = columns[column]
                    threshold_for_devide = best_threshold_for_devide_for_one_column

        return [column_for_devide, threshold_for_devide]

    def build_tree(self, X, y, depth):
        if depth >= self.max_depth or len(y) < self.min_samples:
            value = y.mean() if y.nunique() > 1 else y.iloc[0]
            return Node(value=value, class_counts=y.value_counts().to_dict())

        column, threshold = self.search_best_threshold(X, y)
        if column is None:
            value = y.mean() if y.nunique() > 1 else y.iloc[0]
            return Node(value=value, class_counts=y.value_counts().to_dict())

        left_mask = X[column] <= threshold
        right_mask = X[column] > threshold

        left_child = self.build_tree(X[left_mask], y[left_mask], depth + 1)
        right_child = self.build_tree(X[right_mask], y[right_mask], depth + 1)

        return Node(feature=column, threshold=threshold, left=left_child, right=right_child)

    def fit(self, X, y):
        self.root = self.build_tree(X, y, depth=0)

    def predict(self, X):
        # Создаем копию X
        X = X.copy()

        # Применяем дерево для каждого примера
        predictions = []
        for _, row in X.iterrows(): # бежим по циклу по всем строкам X ("_" - индекс строки, "row" - Series)
            node = self.root # узел это 
            # Идем по дереву пока не пустые и спускаемся в самый "младший" лист
            while node.left is not None and node.right is not None:
                
                # находим  столбец и проверяем с какой стороны он находится слева или справа
                
                if row[node.feature] <= node.threshold: 
                    node = node.left
                else:
                    node = node.right
            # Находим вероятность для листа
            predictions.append(node.value)  # среднее значение в этом листе

        return np.array(predictions)

In [23]:
%%time
# 1. Создание модели дерева решений
my_model_TreeRegressor = My_Decision_Tree_Regressor(max_depth=5)

# 2. Обучение модели
my_model_TreeRegressor.fit(X,y)

CPU times: total: 2.73 s
Wall time: 2.72 s


In [24]:
%%time
# 3. Получение предсказаний
my_y_pred = my_model_TreeRegressor.predict(X)

CPU times: total: 4.75 s
Wall time: 4.81 s


In [25]:
%%time
# 1. Создание модели дерева решений
model_TreeRegressor = DecisionTreeRegressor(max_depth=5)

# 2. Обучение модели
model_TreeRegressor.fit(X,y)

CPU times: total: 125 ms
Wall time: 114 ms


DecisionTreeRegressor(max_depth=5)

In [26]:
# 3. Получение предсказаний
y_pred = model_TreeRegressor.predict(X)

In [27]:
my_mse = mean_squared_error(y, my_y_pred)
mse = mean_squared_error(y, y_pred)

print(f'Mean Squared Error: {my_mse}')
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 486152771.1051127
Mean Squared Error: 483139990.6263404


#### Результат схожий